In [100]:
#BeautifulSoap for scraping
from bs4 import BeautifulSoup
# import webdriver which creates automated selenium sessions
from selenium import webdriver
#import and setup libraries which make explicit wait possible
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#import NoSuchException To handle cases where it tries to break the code
from selenium.common.exceptions import NoSuchElementException
#import TimeOutException To handle cases where explicit wait fails
from selenium.common.exceptions import TimeoutException
#import ElementClickInterceptedException in cases where state of button has changed after clicking
from selenium.common.exceptions import ElementClickInterceptedException
#time library
import time
#to scroll popup window 
from selenium.webdriver.common.keys import Keys
#import date
from datetime import date
#import libraries for plotting graph
import  matplotlib.pyplot as plt
#import pandas
import pandas as pd 

In [204]:
class InstaBot2:
    def __init__(self):
        # start a selenium session and visit the insta login page
        self.driver=webdriver.Chrome(executable_path='C:\Program Files\chromedriver.exe')
        self.checkedStory=False
        time.sleep(3)
        return 
    def signIn(self,username,password):
        # fetch the input boxes and pass required info and click login button to login
        self.driver.get('https://www.instagram.com/accounts/login/')
        time.sleep(5)
        username_inputbox=self.driver.find_element_by_name('username')
        username_inputbox.send_keys(username)
        username_passwordbox=self.driver.find_element_by_name('password')
        username_passwordbox.send_keys(password)
        login_button=self.driver.find_element_by_css_selector('button.sqdOP.L3NKy.y3zKF')
        login_button.click()
        time.sleep(3)
        #if save password prompt pops up handle that
        try:
            wait=WebDriverWait(self.driver,10)
            wait.until(EC.presence_of_element_located((By.XPATH,"//div[contains(@class,'DPiy6')]")))
            not_now_button=self.driver.find_element_by_xpath("//div[@class='cmbtv']/button")
            not_now_button.click()
        except TimeoutException:
            print("Skipped Save Password Popup Or It did not appear")
        #if allow notifications popup appears 
        try:
            wait=WebDriverWait(self.driver,10)
            wait.until(EC.presence_of_element_located((By.XPATH,"//div[@class='_1XyCr']/div")))
            not_now_button_notifications=self.driver.find_element_by_xpath('//div[@class="mt3GC"]/button[2]')
            not_now_button_notifications.click()
        except TimeoutException:
            print("Skipped Turn On Notifications Or It did not appear")
        return
    def searchForKeywordInSearchBar(self,keyword):
        time.sleep(3)
        #type food in the search bar
        insta_main_page_searcbar=self.driver.find_element_by_css_selector('input.XTCLo.x3qfX')
        insta_main_page_searcbar.send_keys(keyword)
        wait=WebDriverWait(self.driver,10)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'div.fuqBx')))
        time.sleep(3)
        return 
    def searchAndOpenProfile(self,profile_name):
        #searching and opening a profile using selenium
        time.sleep(3)
        self.searchForKeywordInSearchBar(profile_name)
        profile_link=self.driver.find_element_by_css_selector("a.-qQT3")
        profile_link.click()
        time.sleep(3)
        return
    def followAPage(self,profile_name):
        time.sleep(3)
        self.searchAndOpenProfile(profile_name)
        #Follow the opened page
        try:
            follow_button=self.driver.find_element_by_xpath('//span[contains(@class,"FLeXg")]/span/button')
            follow_button.click()
        #In case you are already following the page
        except:
            print("Already Following")
    def unFollowAPage(self,profile_name):
        #Unfollow the page
        time.sleep(3)
        self.searchAndOpenProfile(profile_name)
        try:
            unfollow_button=self.driver.find_element_by_css_selector('button._5f5mN.-fzfL._6VtSN.yZn4P')
            unfollow_button.click()
            unfollow_button=self.driver.find_element_by_css_selector('button.aOOlW.-Cab_')
            unfollow_button.click()
        except:
            print("Already Unfollowed, You are not Following this page")
    def fetchFollowersList(self):
        home_button=self.driver.find_element_by_xpath('//div[contains(@class,"Fifk5")]/div/a')
        time.sleep(3)
        home_button.click()
        profile_pic_icon=self.driver.find_element_by_xpath('//div[contains(@class,"Fifk5")]/span/img')
        profile_pic_icon.click()
        goto_profile=self.driver.find_element_by_css_selector('a.-qQT3')
        goto_profile.click()
        time.sleep(3)
        #generate followers dict
        followers_button=self.driver.find_elements_by_xpath('//ul/li/a')
        followers_button[0].click()
        time.sleep(3)
        followers_list=self.driver.find_elements_by_xpath('//li/div/div/div/div/div/span/a')
        page_followers_dict={}
        for element in followers_list:
            page_followers_dict[element.text]=True
        time.sleep(2)
        self.driver.back()
        time.sleep(2)
        return len(page_followers_dict)
    def highestNumberOfFollowersAmongstFoodHandles(self):
        top10SearchResponseList=self.searchForAllLeadingFoodHandles()
        numberOfFollowersOfEachPage=[]
        for idx_of_profile,profile_name in enumerate(top10SearchResponseList):
            self.searchAndOpenProfile(profile_name)
            number_of_followers=self.fetchFollowersList()
            numberOfFollowersOfEachPage.append([idx_of_profile,number_of_followers])
        top5list=sorted(numberOfFollowersOfEachPage,key= lambda item: item[1])
        print(top5list)
        return
    def getListOfTop10FoodHandles(self):
        self.searchForKeywordInSearchBar('food')
        time.sleep(2)
        # Search for all top food handles that appear after searching for food 
        data=BeautifulSoup(self.driver.page_source,'lxml')
        parentOfListOfFoodHandles=data.select('div.fuqBx')
        list_=[]
        listOfFoodHandles=parentOfListOfFoodHandles[0].contents
        for element in listOfFoodHandles:
            textElement=str(element.select('div._7UhW9.xLCgt.qyrsm.KV-D4.uL8Hv')[0].string)
            if '#' in textElement:
                continue
            if textElement=="food":
                continue
            list_.append(textElement)
        listOfTop10foodhandles=list_[:10]
        return listOfTop10foodhandles
    def searchForAllLeadingFoodHandlesAndFindTop5WithMostFollowers(self):
        listOfTop10foodhandles=self.getListOfTop10FoodHandles()
        followers_count=[]
        for i in range(len(listOfTop10foodhandles)):
            time.sleep(3)
            try:
                searcbar=self.driver.find_element_by_css_selector('div.pbgfb.Di7vw')
                time.sleep(3)
                searcbar.click()
                time.sleep(2)
            except:
                print("")
            cross_button=self.driver.find_element_by_class_name('coreSpriteSearchClear')
            cross_button.click()
            time.sleep(2)
            #type food in the search bar
            insta_main_page_searcbar=self.driver.find_element_by_css_selector('input.XTCLo.x3qfX')
            insta_main_page_searcbar.send_keys(listOfTop10foodhandles[i])
            wait=WebDriverWait(self.driver,10)
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'div.fuqBx')))
            time.sleep(3)
            profile_link=self.driver.find_element_by_css_selector("a.-qQT3")
            profile_link.click()
            time.sleep(3)
            listConatiningFollowers=self.driver.find_elements_by_xpath('//ul/li/a/span')
            time.sleep(2)
            followers=int("".join(listConatiningFollowers[0].get_attribute('title').split(',')))
            followers_count.append([followers,listOfTop10foodhandles[i]])
        home_button=self.driver.find_element_by_xpath('//div[contains(@class,"Fifk5")]/div/a')
        time.sleep(3)
        home_button.click()
        followers_count=sorted(followers_count,key=lambda item:item[0],reverse=True)
        followers_count=followers_count[:5]
        return followers_count
    def findNumberOfPostsInPastThreeDaysByLeadingFoodHandles(self):
        foodHandles=self.getListOfTop10FoodHandles()
        foodHandles=foodHandles[:5]
        profile_vs_posts=[]
        for profile in foodHandles:
            self.searchAndOpenProfile(profile)
            self.driver.execute_script('window.scrollBy(0,2000)')
            time.sleep(8)
            #Fetching posts to check number of them posted in past 3 days
            listOfPosts=self.driver.find_elements_by_xpath('//article/div/div//div[contains(@class,"v1Nh3")]')
            count_of_posts_in_past3Days=0
            for i in range(len(listOfPosts)):
                div=listOfPosts[i].click()
                time.sleep(5)
                d=self.driver.find_element_by_css_selector('time._1o9PC.Nzb55')
                a=d.get_attribute('datetime')
                dateObtained=a[:10].split('-')
                today=date.today().strftime("%Y/%m/%d").split("/")
                postedInPast3days=(dateObtained[0]==today[0]) and (abs(int(dateObtained[1])-int(today[1]))==1 or abs(int(dateObtained[1])-int(today[1]))==0) and abs(int(dateObtained[2])-int(today[2]))<3
                if not postedInPast3days:
                    break
                count_of_posts_in_past3Days+=1
                close_btn=self.driver.find_element_by_xpath('//div[contains(@class,"fm1AK")]/button')
                close_btn.click()
                time.sleep(3)
            close_btn=self.driver.find_element_by_xpath('//div[contains(@class,"fm1AK")]/button')
            close_btn.click()
            profile_vs_posts.append([profile,count_of_posts_in_past3Days])    
        return profile_vs_posts
    def plotGraphToShowFrequencyOfPosting(self):
        posts=self.findNumberOfPostsInPastThreeDaysByLeadingFoodHandles()
        profile_name=[i[0] for i in posts]
        number_of_posts=[i[1] for i in posts]
        plt.scatter(profile_name,number_of_posts)
        plt.xticks(rotation=45)
        plt.show()
    def fetchHashtagsFromLast10PostsandSaveToCsvAndDrawPieChart(self):
        foodHandles=self.getListOfTop10FoodHandles()
        foodHandles=foodHandles[:5]
        profile_vs_hastags={}
        for profile in foodHandles:
            self.searchAndOpenProfile(profile)
            self.driver.execute_script('window.scrollBy(0,2000)')
            time.sleep(8)
            #Fetching posts to check number of them posted in past 3 days
            listOfPosts=self.driver.find_elements_by_xpath('//article/div/div//div[contains(@class,"v1Nh3")]')
            tenPosts=listOfPosts[:10]
            hastagWords=[]
            for i in range(len(tenPosts)):
                div=listOfPosts[i].click()
                time.sleep(5)
                hastags=self.driver.find_elements_by_css_selector('a.xil3i')
                for hastag in hastags:
                    hastagWords.append(hastag.text)
                close_btn=self.driver.find_element_by_xpath('//div[contains(@class,"fm1AK")]/button')
                close_btn.click()
                time.sleep(3)
            profile_vs_hastags[profile]=hastagWords
        frequency={}
        for hastags in profile_vs_hastags:
            value=d[hastags]
        for hastag in value:
            if hastag=='':
                continue
            frequency[hastag]=frequency.get(hastag,0)+1
        df = pd.DataFrame(list(frequency.items()))
        df.columns=['Word','Frequency']
        df.to_csv('wordfrequency.csv') 
        fr=sorted(frequency.items(),key=lambda item:item[1],reverse=True)
        print("Top 20 Most Popular Hastags are")
        for i in range(20):
            print(fr[i][0],fr[i][1])
        top5Hastags=fr[:5]
        labels=[i[0] for i in top5Hastags]
        vals=[i[1] for i in top5Hastags]
        plt.pie(vals,labels=labels,autopct="%.2f")
        plt.show()
    def averageFollowerToLikeRatio(self):
        foodHandles=self.getListOfTop10FoodHandles()
        foodHandles=foodHandles[:5]
        likes_vs_handles={}
        for profile in foodHandles:
            self.searchAndOpenProfile(profile)
            self.driver.execute_script('window.scrollBy(0,2000)')
            time.sleep(8)
            #Fetching posts to check number of them posted in past 3 days
            listOfPosts=self.driver.find_elements_by_xpath('//article/div/div//div[contains(@class,"v1Nh3")]')
            followers=self.driver.find_element_by_xpath('//li/a/span')
            followersI=followers.get_attribute('title')
            followers_count=int("".join(followersI.split(",")))
            count=0
            likesInPosts=0
            for i in range(len(listOfPosts)):
                div=listOfPosts[i].click()
                time.sleep(5)
                try:
                    likesOnPost=self.driver.find_elements_by_partial_link_text('likes')
                    if likesOnPost==[]:
                        close_btn=self.driver.find_element_by_xpath('//div[contains(@class,"fm1AK")]/button')
                        close_btn.click()
                        continue
                    likes=int("".join(likesOnPost[0].text.split(" ")[0].split(",")))
                    likesInPosts+=likes
                    count+=1
                    close_btn=self.driver.find_element_by_xpath('//div[contains(@class,"fm1AK")]/button')
                    close_btn.click()
                    if count==10:
                        break
                    time.sleep(3)
                except:
                    close_btn=self.driver.find_element_by_xpath('//div[contains(@class,"fm1AK")]/button')
                    close_btn.click()
                    continue
            likes_vs_handles[profile]=[likesInPosts,followers_count]
        likes_on_top10posts=[likes_vs_handles[key][0] for key in likes_vs_handles]
        followers=[likes_vs_handles[key][1] for key in likes_vs_handles]
        average=[likes_on_top10posts[i]/followers[i] for i in range(5)]        
        plt.title("followers_vs_likes")
        plt.xlabel('Followers')
        plt.ylabel('RatioOfLikesToFollowers')
        plt.xticks(rotation=45)
        plt.bar(foodHandles,average,edgecolor="black",width=1)
        plt.show()